# COI Data Preparation: Refinement and Development

**Objective:** Refine the data preparation pipeline for the COI gene, the primary barcode for animal identification, using the BOLD database.

**Methodology:**
1.  Work from a small, manageable sample of the full BOLD database.
2.  Develop a new taxonomy parser specifically designed for the pipe-separated (`|`) format of BOLD FASTA headers.
3.  Apply the full data cleaning and feature engineering workflow.
4.  Save the final, uniquely named artifacts for the COI pipeline.

In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from tqdm.auto import tqdm
from pathlib import Path
import sys

# Set up project path
project_root = Path.cwd().parent

# --- Verification Step ---
print(f"Project Root: {project_root}")

Project Root: C:\Users\jampa\Music\atlas-v3


In [3]:
# --- Define Core Directories ---
RAW_DATA_DIR = project_root / "data" / "raw"
PROCESSED_DATA_DIR = project_root / "data" / "processed"
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

# --- Define COI Specific File Paths ---

# --- FIX: Use the exact filename of the downloaded BOLD database ---
# Path to the full, original BOLD database file
FULL_BOLD_PATH = RAW_DATA_DIR / "BOLD_Public.29-Aug-2025.fasta"

# Path to the small sample file we will create for development
SAMPLE_BOLD_PATH = RAW_DATA_DIR / "BOLD_sample_10k.fasta"

# --- Verification Step ---
if FULL_BOLD_PATH.exists():
    print("Source BOLD database found.")
    print(f"  - Location: {FULL_BOLD_PATH}")
else:
    print(f"ERROR: The source BOLD database was not found at the expected location.")
    print(f"  - Expected: {FULL_BOLD_PATH}")
    print("Please ensure the file is downloaded and correctly named in the 'data/raw' directory.")

Source BOLD database found.
  - Location: C:\Users\jampa\Music\atlas-v3\data\raw\BOLD_Public.29-Aug-2025.fasta


In [4]:
# =============================================================================
# STEP 1: CREATE A DEVELOPMENT SAMPLE FROM BOLD DATA
# =============================================================================
#
# OBJECTIVE:
#   To create a smaller 10,000-sequence sample from the full BOLD database.
#
# RATIONALE:
#   Working with a smaller sample allows for rapid, interactive development
#   and debugging of the subsequent parsing and cleaning steps without the
#   long processing times required for the full dataset. This script is
#   designed to be run only once; if the sample file is found, this
#   step will be skipped.
#
# =============================================================================

# --- Configuration ---
SAMPLE_SIZE = 10000

# --- Main Logic ---
# This check prevents us from re-running this process unnecessarily.
if not SAMPLE_BOLD_PATH.exists():
    print(f"Creating a sample of {SAMPLE_SIZE} sequences from the BOLD database...")
    
    sample_records = []
    # Open the full BOLD database file for reading
    with open(FULL_BOLD_PATH, "r") as handle_in:
        # Create an efficient iterator over the FASTA records
        records_iterator = (record for record in SeqIO.parse(handle_in, "fasta"))
        
        # Use tqdm to show a progress bar while iterating
        for i, record in tqdm(enumerate(records_iterator), total=SAMPLE_SIZE, desc="Sampling BOLD DB"):
            # Stop once we have collected the desired number of samples
            if i >= SAMPLE_SIZE:
                break
            sample_records.append(record)
            
    # Write the collected records to our new sample file
    with open(SAMPLE_BOLD_PATH, "w") as handle_out:
        SeqIO.write(sample_records, handle_out, "fasta")
        
    print(f"\nSuccessfully created sample file with {len(sample_records)} sequences.")
    print(f"   Location: {SAMPLE_BOLD_PATH}")
else:
    print(f"BOLD sample file already exists. No action needed.")
    print(f"   Location: {SAMPLE_BOLD_PATH}")

Creating a sample of 10000 sequences from the BOLD database...


Sampling BOLD DB:   0%|          | 0/10000 [00:00<?, ?it/s]


Successfully created sample file with 10000 sequences.
   Location: C:\Users\jampa\Music\atlas-v3\data\raw\BOLD_sample_10k.fasta


In [5]:
# =============================================================================
# STEP 2: PARSE TAXONOMY FROM BOLD SAMPLE DATA
# =============================================================================
#
# OBJECTIVE:
#   To develop a new taxonomy parser specifically for the pipe-separated (`|`)
#   format of the BOLD database FASTA headers and apply it to our sample data.
#
# RATIONALE:
#   Each data source has a unique format. A custom parser is required to
#   correctly extract the taxonomic ranks (Phylum, Class, Order, etc.) and
#   transform the raw text into a structured, usable format. The successful
#   creation of a clean DataFrame is the most critical step in this notebook.
#
# EXAMPLE BOLD HEADER:
#   >ABZ-9023|Chordata|...|Homo|Homo sapiens
#
# =============================================================================

# --- A list to hold our structured data ---
parsed_data = []

# --- Define the BOLD-specific parsing function ---
def parse_bold_taxonomy(description):
    """
    Parses the pipe-separated taxonomy from a BOLD FASTA header.
    This function assumes a common, though not perfectly consistent,
    structure of Phylum|Class|Order|Family|Genus|Species after the ID.
    """
    # Initialize a dictionary with all ranks set to None
    parsed_ranks = {
        'kingdom': None, 'phylum': None, 'class': None, 'order': None,
        'family': None, 'genus': None, 'species': None
    }
    
    try:
        # BOLD does not typically list "Kingdom", but since it's the animal
        # barcode, we can often infer it as "Metazoa" or "Animalia".
        # We will leave it as None for now to rely only on provided data.
        
        parts = description.split('|')
        
        # Assign ranks based on their typical position after the ID part.
        # These checks prevent errors if a header is malformed or short.
        if len(parts) > 1: parsed_ranks['phylum'] = parts[1]
        if len(parts) > 2: parsed_ranks['class'] = parts[2]
        if len(parts) > 3: parsed_ranks['order'] = parts[3]
        if len(parts) > 4: parsed_ranks['family'] = parts[4]
        if len(parts) > 5: parsed_ranks['genus'] = parts[5]
        if len(parts) > 6: parsed_ranks['species'] = parts[6]

    except Exception:
        # If any unexpected error occurs during parsing, we pass silently,
        # leaving the ranks as None. This makes the function robust.
        pass
        
    return parsed_ranks

# --- Apply the parser to our sample data ---
print("Applying BOLD taxonomy parser to sample data...")
# Loop through the records in our sample file
with open(SAMPLE_BOLD_PATH, "r") as handle:
    for record in tqdm(SeqIO.parse(handle, "fasta"), total=SAMPLE_SIZE, desc="Parsing BOLD headers"):
        # The entire description line contains the taxonomy
        taxonomy_dict = parse_bold_taxonomy(record.description)
        
        # Store the essential sequence information
        taxonomy_dict['id'] = record.id
        taxonomy_dict['sequence'] = str(record.seq)
        
        parsed_data.append(taxonomy_dict)

# --- Create and Verify the DataFrame ---
# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(parsed_data)

print(f"\nParsing complete. Created a DataFrame with {len(df)} rows.")

# Display the first 5 rows for a preliminary check
print("\nPreview of the structured data (first 5 rows):")
display(df.head())

# Display 5 random rows to check for consistency across the dataset
print("\nPreview of the structured data (5 random rows):")
display(df.sample(5))

Applying BOLD taxonomy parser to sample data...


Parsing BOLD headers:   0%|          | 0/10000 [00:00<?, ?it/s]


Parsing complete. Created a DataFrame with 10000 rows.

Preview of the structured data (first 5 rows):


,kingdom,phylum,class,order,family,genus,species,id,sequence
0,None,COI-5P,Australia,"Animalia,Arthropoda,Insecta,Lepidoptera,Geomet...",None,None,None,"AANIC003-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCTGGTATAATTGGAACT...
1,None,COI-5P,Australia,"Animalia,Arthropoda,Insecta,Lepidoptera,Geomet...",None,None,None,"AANIC101-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCTGGTATAATTGGAACT...
2,None,COI-5P,Australia,"Animalia,Arthropoda,Insecta,Lepidoptera,Geomet...",None,None,None,"AANIC102-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCTGGTATAATTGGAACT...
3,None,COI-5P,Australia,"Animalia,Arthropoda,Insecta,Lepidoptera,Geomet...",None,None,None,"AANIC104-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCAGGTATAATTGGAACT...
4,None,COI-5P,Australia,"Animalia,Arthropoda,Insecta,Lepidoptera,Geomet...",None,None,None,"AANIC105-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCAGGTATAATTGGAACT...



Preview of the structured data (5 random rows):


,kingdom,phylum,class,order,family,genus,species,id,sequence
3154,None,None,Australia,"Animalia,Arthropoda,Insecta,Psocodea,None,None...",None,None,None,"AACTA1346-20|None|Australia|Animalia,Arthropod...",
1679,None,COI-5P,Mexico,"Animalia,Arthropoda,Insecta,Diptera,Psychodida...",None,None,None,"AAASF045-18|COI-5P|Mexico|Animalia,Arthropoda,...",AACCCTTTATTTTATCTTTGGAGCCTGATCAGGAATAGTAGGAACA...
1456,None,COI-5P,Austria,"Animalia,Arthropoda,Insecta,Coleoptera,Staphyl...",None,None,None,"AALCO078-16|COI-5P|Austria|Animalia,Arthropoda...",AACATTGTATTTCATTTTTGGGGCTTGAGCAGGAATAGTAGGTACA...
9385,None,None,Australia,"Animalia,Arthropoda,Insecta,Diptera,None,None,...",None,None,None,"AACTA7171-20|None|Australia|Animalia,Arthropod...",
5379,None,COI-5P,Australia,"Animalia,Arthropoda,Insecta,Diptera,Cecidomyii...",None,None,None,"AACTA3424-20|COI-5P|Australia|Animalia,Arthrop...",CATTATATTTTATATTTGGAATTTGATCTGGAATATTAGGTACCTC...
